# Vacation Py

In [1]:
#!pip install requests
#!pip install gmaps

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Store Part I results into DataFrame

In [2]:
#get csv file
csv_file = "cities.csv"

#read csv file
cities_df = pd.read_csv(csv_file)

#drop na in file
cities_df.dropna(inplace=True)

#print results
cities_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


# Humidity Heatmap

In [3]:
#getting info from API key
gmaps.configure(api_key=g_key)
fig_map = gmaps.figure()

#Setting Lat and Lng as locations
locations = cities_df[['Lat', 'Lng']].astype(float)

#make humidity list
humd_list = cities_df['Humidity'].astype(int)

#create a humity heatmap layer
fig_map = gmaps.figure(map_type='TERRAIN')

#Find locations 
heatmap_layer = gmaps.heatmap_layer(locations, weights=humd_list, dissipating=False, max_intensity=100, point_radius = 2)

#add layers
fig_map.add_layer(heatmap_layer)

#Show all layers
fig_map

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria

In [4]:
#drop cities with a max temperature lower than 80 degrees but higher than 70.
cities_df = cities_df.drop(cities_df[(cities_df['Max Temp']>=80)].index)
cities_df = cities_df.drop(cities_df[(cities_df['Max Temp']<=70)].index)

#drop cities with wind speed less than 10 mph.
cities_df = cities_df.drop(cities_df[(cities_df['Wind Speed']>=10)].index)

#drop cities with cloudiness.
hotel_df = cities_df.drop(cities_df[(cities_df['Cloudiness']>0)].index)

#print results
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


# Hotel Map

In [5]:
#add in Hotel info to dataframe
hotel_df['Hotel_Name'] = ""
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel_Name
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,


In [6]:
#make city list
city= list(hotel_df['City'])
city

['ponta do sol', 'cidreira', 'morondava', 'vaini', 'nador', 'mogok', 'birjand']

In [7]:
#params dictionary 
params = {'radius' :5000,
         'keyword': 'lodging',
         'key' : g_key}


#for loop to find hotel info for the ideal conditions on dataframe
for index, row in hotel_df.iterrows():
    #getting Lat and Lng from df
    lat = row['Lat']
    lng = row['Lng']
    
    #change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    #getting url
    url ="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #print (params ['key'])
    response = requests.get(url, params=params)
    
    #convert to json
    results = response.json()
   
    #hotels = results['results']
    try:
        print(f"{results['results'][0]['name']} is the closest to lodging.")
        hotel_df.loc[index, 'Hotel_Name'] = results['results'][0]['name']
    except (KeyError, IndexError):
        print("Error found skipping")


Kanto Island Inn Suites & Spa is the closest to lodging.
Hotel Castelo is the closest to lodging.
Palissandre Cote Ouest resort & SPA is the closest to lodging.
Dandeli Mist Jungle Stay is the closest to lodging.
Hotel Marchica Lagoon Resort, Nador Morocco is the closest to lodging.
Mount Mogok Hotel is the closest to lodging.
مجتمع آریا is the closest to lodging.


In [8]:
hotel_df = hotel_df.drop(hotel_df[(hotel_df['Hotel_Name']=="")].index)
#display results
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel_Name
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Kanto Island Inn Suites & Spa
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Castelo
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Palissandre Cote Ouest resort & SPA
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Dandeli Mist Jungle Stay
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,"Hotel Marchica Lagoon Resort, Nador Morocco"


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
#create a hotel Marker layer
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig_map.add_layer(marker_layer)

#display map
fig_map

Figure(layout=FigureLayout(height='420px'))